# 전처리, 키워드 추출, 감정분석

## 1. 전처리

### 1) Import Data

In [ ]:
# pip install konlpy

In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from nltk.tokenize import sent_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BITAminNLP /대구관광지리뷰.csv')
df

In [ ]:
# pip install soynlp

### 2) Cleaning and Normalization

In [ ]:
import re
from soynlp.normalizer import repeat_normalize

# 정제 및 정규화 함수 정의
def clean_and_normalize(text):
    # 특수 문자와 숫자 제거
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)

    # 중복되는 자음/모음 제거 (예: 'ㅋㅋㅋㅋ' -> 'ㅋㅋ')
    text = repeat_normalize(text, num_repeats=2)

    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# '리뷰' 컬럼에 대해 전처리 적용
df['리뷰'] = df['리뷰'].apply(clean_and_normalize)

# 전처리된 데이터 확인
df.head()

### 3) Tokenization

### STOPWORDS

In [ ]:
stopwords = '/content/drive/MyDrive/BITAminNLP /stopword.txt'
stopwords

In [ ]:
with open(stopwords, 'r', encoding='utf-8') as f:
    stopword = set(f.read().split())

stopword

### 형태소 기반 토큰화

In [ ]:
from konlpy.tag import Okt

okt = Okt()
df['리뷰_형태소토큰화_Okt'] = df['리뷰'].apply(okt.morphs)

In [ ]:
# 불용어 제거 함수 정의 (토큰화된 데이터용)
def remove_stopwords(tokenized_text):
    meaningful_words = [word for word in tokenized_text if word not in stopword]  # 불용어가 아닌 단어만 남김
    return meaningful_words  # 리스트 형태로 반환

# 데이터프레임의 토큰화된 리뷰 컬럼에 대해 불용어 제거 적용
df['리뷰_불용어제거'] = df['리뷰_형태소토큰화_Okt'].apply(remove_stopwords)

# 결과 확인
df[['리뷰_형태소토큰화_Okt', '리뷰_불용어제거']].head()

In [ ]:
df.to_csv('형태소토큰화.csv', index=False, encoding='utf-8-sig')
df = df.drop('리뷰_형태소토큰화_Okt',axis=1)
df = df.drop('리뷰_불용어제거',axis=1)

### Subword 기반 토큰화

In [ ]:
# pip install sentencepiece

In [ ]:
import pandas as pd

# 모든 리뷰 텍스트를 하나의 파일로 저장
with open('/content/data.txt', 'w', encoding='utf-8') as f:
    for review in df['리뷰']:
        f.write(review + '\n')

In [ ]:
import sentencepiece as spm

# SentencePiece 모델 학습
spm.SentencePieceTrainer.Train('--input=/content/data.txt --model_prefix=spm --vocab_size=32000 --model_type=bpe')

# 모델 파일이 spm.model로 저장됩니다.

In [ ]:
sp = spm.SentencePieceProcessor()
sp.load('/content/spm.model')

# 예시: 첫 번째 리뷰를 토큰화
tokenized_review = sp.encode_as_pieces(df['리뷰'][0])
print(tokenized_review)

# 데이터프레임에 모든 리뷰 토큰화 결과 저장
df['리뷰_Sentence_Piece'] = df['리뷰'].apply(lambda x: sp.encode_as_pieces(x))

In [ ]:
df.head()

In [ ]:
# 불용어 제거 함수 정의 (토큰화된 데이터용)
def remove_stopwords(tokenized_text):
    meaningful_words = [word for word in tokenized_text if word not in stopword]  # 불용어가 아닌 단어만 남김
    return meaningful_words  # 리스트 형태로 반환

# 데이터프레임의 토큰화된 리뷰 컬럼에 대해 불용어 제거 적용
df['리뷰_불용어제거'] = df['리뷰_Sentence_Piece'].apply(remove_stopwords)

# 결과 확인
df[['리뷰_Sentence_Piece', '리뷰_불용어제거']].head()

In [ ]:
# 토큰화된 데이터를 CSV로 저장
df.to_csv('subword 토큰화.csv', index=False, encoding='utf-8-sig')

In [ ]:
df = df.drop('리뷰_Sentence_Piece',axis=1)
df = df.drop('리뷰_불용어제거',axis=1)

### 공백단위 토큰화

In [ ]:
# 공백 단위 토큰화 함수 정의
def split_tokenize(text):
    if isinstance(text, str):  # 텍스트가 문자열인지 확인
        return text.split()
    else:
        return []  # 텍스트가 아닌 경우 빈 리스트 반환

# 공백 단위 토큰화 적용
df['공백단위_토큰화'] = df['리뷰'].apply(split_tokenize)

In [ ]:
# 데이터프레임의 토큰화된 리뷰 컬럼에 대해 불용어 제거 적용
df['리뷰_불용어제거'] = df['공백단위_토큰화'].apply(remove_stopwords)

# 결과 확인
df[['공백단위_토큰화', '리뷰_불용어제거']].head()

In [ ]:
df = df.drop('공백단위_토큰화',axis=1)
df.to_csv('공백 토큰화.csv', index=False, encoding='utf-8-sig')

In [ ]:
df = df.drop('리뷰_불용어제거',axis=1)

### 4) 표제어 추출

In [ ]:
import pandas as pd
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

### subword

In [ ]:
# CSV 파일 로드 (예: 공백 토큰화된 파일 사용)
df = pd.read_csv('/content/drive/MyDrive/BITAminNLP /CSV Files/subword 토큰화.csv')

# 표제어 추출 함수 정의
def lemmatize(text):
    # 형태소 분석을 통해 표제어 추출
    tokens = okt.pos(text, norm=True, stem=True)
    return [word for word, tag in tokens]  # 표제어 리스트로 반환

# 표제어 추출 적용
df['리뷰_표제어추출'] = df['리뷰_불용어제거'].apply(lambda x: ' '.join(lemmatize(x)))

# 결과 확인
print(df[['리뷰_불용어제거', '리뷰_표제어추출']].head())

# 전처리된 파일 저장
df.to_csv('Subword_표제어추출.csv', index=False)

### 공백단위

In [ ]:
df = pd.read_csv('/content/공백 토큰화.csv')

# 표제어 추출 적용
df['리뷰_표제어추출'] = df['리뷰_불용어제거'].apply(lambda x: ' '.join(lemmatize(x)))

# 결과 확인
print(df[['리뷰_불용어제거', '리뷰_표제어추출']].head())

# 전처리된 파일 저장
df.to_csv('공백토큰화_표제어추출.csv', index=False)

### 형태소

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BITAminNLP /CSV Files/형태소토큰화.csv')

# 표제어 추출 적용
df['리뷰_표제어추출'] = df['리뷰_불용어제거'].apply(lambda x: ' '.join(lemmatize(x)))

# 결과 확인
print(df[['리뷰_불용어제거', '리뷰_표제어추출']].head())

# 전처리된 파일 저장
df.to_csv('형태소토큰화_표제어추출.csv', index=False)

## 2. TF-IDF 키워드 추출

### 1) 관광지별 키워드 추출

In [ ]:
# pip install konlpy

In [ ]:
import pandas as pd
from konlpy.tag import Okt
import re
from sklearn.feature_extraction.text import TfidfVectorizer

### subword

In [ ]:
df = pd.read_csv('Subword_표제어추출.csv')
df

In [ ]:
# '전처리된_댓글' 열의 값을 문자열로 변환하고 결측치를 빈 문자열로 채움
df['리뷰_표제어추출'] = df['리뷰_표제어추출'].fillna('').astype(str)

# 사용자 지정 불용어 목록
custom_stop_words = ['싶다', '좋다','가보다', '징쨔', '더욱','이다','보다','해보다','많다',]

# TF-IDF 벡터라이저 초기화 (사용자 지정 불용어 포함)
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stop_words)


# 관광지별 댓글을 그룹화하여 하나의 문자열로 합침
grouped = df.groupby('관광지')['리뷰_표제어추출'].apply(' '.join).reset_index()

# TF-IDF 행렬 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped['리뷰_표제어추출'])

# TF-IDF 점수 매트릭스를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=grouped['관광지'])

# 관광지별 상위 5개 키워드 추출
top_keywords = {}
for idx, row in tfidf_df.iterrows():
    top_keywords[idx] = row.nlargest(10).index.tolist()

# 결과 출력
for tourist_spot, keywords in top_keywords.items():
    print(f'{tourist_spot}의 상위 10개 키워드: {", ".join(keywords)}')

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(list(top_keywords.items()), columns=['관광지', '키워드'])

# csv저장
results_df.to_csv('TF-IDF subword 표제어추출.CSV', index=False, encoding='utf-8-sig')

results_df

### 공백단위

In [ ]:
df = pd.read_csv('공백토큰화_표제어추출.csv')
df

In [ ]:
# '전처리된_댓글' 열의 값을 문자열로 변환하고 결측치를 빈 문자열로 채움
df['리뷰_표제어추출'] = df['리뷰_표제어추출'].fillna('').astype(str)

# 사용자 지정 불용어 목록
custom_stop_words = ['싶다', '좋다','가보다', '징쨔', '더욱','이다','보다','해보다','많다',]

# TF-IDF 벡터라이저 초기화 (사용자 지정 불용어 포함)
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stop_words)


# 관광지별 댓글을 그룹화하여 하나의 문자열로 합침
grouped = df.groupby('관광지')['리뷰_표제어추출'].apply(' '.join).reset_index()

# TF-IDF 행렬 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped['리뷰_표제어추출'])

# TF-IDF 점수 매트릭스를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=grouped['관광지'])

# 관광지별 상위 5개 키워드 추출
top_keywords = {}
for idx, row in tfidf_df.iterrows():
    top_keywords[idx] = row.nlargest(10).index.tolist()

# 결과 출력
for tourist_spot, keywords in top_keywords.items():
    print(f'{tourist_spot}의 상위 10개 키워드: {", ".join(keywords)}')

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(list(top_keywords.items()), columns=['관광지', '키워드'])

# csv저장
results_df.to_csv('TF-IDF 공백 표제어 추출.CSV', index=False, encoding='utf-8-sig')

results_df

### 형태소

In [ ]:
df = pd.read_csv('형태소토큰화_표제어추출.csv')
df

In [ ]:
# '전처리된_댓글' 열의 값을 문자열로 변환하고 결측치를 빈 문자열로 채움
df['리뷰_표제어추출'] = df['리뷰_표제어추출'].fillna('').astype(str)

# 사용자 지정 불용어 목록
custom_stop_words = ['싶다', '좋다','가보다', '징쨔', '더욱','이다','보다','해보다','많다',]

# TF-IDF 벡터라이저 초기화 (사용자 지정 불용어 포함)
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stop_words)


# 관광지별 댓글을 그룹화하여 하나의 문자열로 합침
grouped = df.groupby('관광지')['리뷰_표제어추출'].apply(' '.join).reset_index()

# TF-IDF 행렬 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped['리뷰_표제어추출'])

# TF-IDF 점수 매트릭스를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=grouped['관광지'])

# 관광지별 상위 5개 키워드 추출
top_keywords = {}
for idx, row in tfidf_df.iterrows():
    top_keywords[idx] = row.nlargest(10).index.tolist()

# 결과 출력
for tourist_spot, keywords in top_keywords.items():
    print(f'{tourist_spot}의 상위 10개 키워드: {", ".join(keywords)}')

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(list(top_keywords.items()), columns=['관광지', '키워드'])

# csv저장
results_df.to_csv('형태소토큰화_표제어추출.CSV', index=False, encoding='utf-8-sig')

results_df

### 2) 키워드별 관광지 분류(키워드 10개)

### 공백 표제어 추출 (최종 선택 방법)

In [ ]:
import pandas as pd
from collections import Counter
import ast

# 파일 경로
file_path = '/content/TF-IDF TEST.CSV'

# 데이터 불러오기
data = pd.read_csv(file_path)

# 불용어 리스트 파일 경로
stopword_file_path = 'stopword.txt'

# 텍스트 파일에서 불용어 리스트 불러오기
with open(stopword_file_path, 'r', encoding='utf-8') as file:
    stopwords = set(file.read().splitlines())

# 추가할 불용어 리스트
additional_stopwords = {'하다', '있다', '너무', '대구', '에서', '오다', '올라가다'}
stopwords.update(additional_stopwords)

def clean_keywords(keywords):
    # 문자열을 리스트로 변환하고 불용어 제거
    keywords_list = ast.literal_eval(keywords)
    return [word.lower() for word in keywords_list if word.lower() not in stopwords]

# 키워드를 리스트로 변환하고 불용어 제거
data['키워드'] = data['키워드'].apply(clean_keywords)

# 유의어 처리 및 '군위' 제외
synonyms = {
    '공원': '산책','사진': '전망대',
    '놀다' : '체험', '먹다': '시장'
}

def replace_synonyms(keywords):
    return [synonyms.get(word, word) for word in keywords if word.lower() != '군위']

data['키워드'] = data['키워드'].apply(replace_synonyms)

# 모든 키워드를 하나의 리스트로 합치기
all_keywords = [keyword for sublist in data['키워드'] for keyword in sublist]

# 키워드 빈도 계산
keyword_counts = Counter(all_keywords)

# 상위 N개의 키워드 추출
top_n = 10
top_keywords = [keyword for keyword, _ in keyword_counts.most_common(top_n)]

# 상위 N개 키워드에 해당하는 관광지 리스트 매핑
keyword_to_places = {keyword: [] for keyword in top_keywords}

for keyword in top_keywords:
    places = data.loc[data['키워드'].apply(lambda x: keyword in x), '관광지 이름'].unique().tolist()
    keyword_to_places[keyword] = sorted(places)  # 관광지 리스트를 정렬하여 저장

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame([(keyword, ', '.join(places)) for keyword, places in keyword_to_places.items()],
                         columns=['키워드', '관광지 리스트'])

# 결과 저장
result_file_path = '공백표제어_관광지리스트.csv'
result_df.to_csv(result_file_path, index=False, encoding='utf-8-sig')  # UTF-8 BOM 추가

# 결과 파일 경로 출력
print("결과 파일 경로:", result_file_path)

### 형태소 표제어 추출

In [ ]:
# 파일 경로
file_path = './data/형태소토큰화_표제어추출.CSV'

# 데이터 불러오기
data = pd.read_csv(file_path)

# 불용어 리스트 파일 경로
stopword_file_path = './data/stopword.txt'

# 텍스트 파일에서 불용어 리스트 불러오기
with open(stopword_file_path, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# 추가할 불용어 리스트
additional_stopwords = ['하다', '있다', '너무', '대구']

# 기존 불용어 리스트에 추가 불용어를 합치기
stopwords.extend(additional_stopwords)

# 키워드를 리스트로 변환하고 불용어 제거
data['키워드'] = data['키워드'].apply(lambda x: [word for word in ast.literal_eval(x) if word not in stopwords])

# 모든 키워드를 하나의 리스트로 합치기
all_keywords = [keyword for sublist in data['키워드'] for keyword in sublist]

# 키워드 빈도 계산
keyword_counts = Counter(all_keywords)

# 가장 많이 언급된 상위 10개 키워드 추출
top_keywords = [keyword for keyword, _ in keyword_counts.most_common(10)]

# 상위 10개 키워드에 해당하는 관광지 리스트 매핑
keyword_to_places = {keyword: [] for keyword in top_keywords}

for keyword in top_keywords:
    places = data.loc[data['키워드'].apply(lambda x: keyword in x), '관광지명'].unique().tolist()
    keyword_to_places[keyword] = places

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame([(keyword, ', '.join(places)) for keyword, places in keyword_to_places.items()], 
                         columns=['키워드', '관광지 리스트'])

# 결과 저장
result_file_path = './data/형태소토큰화_관광지리스트.csv'
result_df.to_csv(result_file_path, index=False)

# 결과 파일 경로 출력
result_file_path

### 서브워드 표제어 추출

In [ ]:
# 파일 경로
file_path = './data/TF-IDF subword 표제어추출.CSV'

# 데이터 불러오기
data = pd.read_csv(file_path)

# 불용어 리스트 파일 경로
stopword_file_path = './data/stopword.txt'

# 텍스트 파일에서 불용어 리스트 불러오기
with open(stopword_file_path, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# 추가할 불용어 리스트
additional_stopwords = ['하다', '있다', '너무', '대구']

# 기존 불용어 리스트에 추가 불용어를 합치기
stopwords.extend(additional_stopwords)

# 키워드를 리스트로 변환하고 불용어 제거
data['키워드'] = data['키워드'].apply(lambda x: [word for word in ast.literal_eval(x) if word not in stopwords])

# 모든 키워드를 하나의 리스트로 합치기
all_keywords = [keyword for sublist in data['키워드'] for keyword in sublist]

# 키워드 빈도 계산
keyword_counts = Counter(all_keywords)

# 가장 많이 언급된 상위 10개 키워드 추출
top_keywords = [keyword for keyword, _ in keyword_counts.most_common(10)]

# 상위 10개 키워드에 해당하는 관광지 리스트 매핑
keyword_to_places = {keyword: [] for keyword in top_keywords}

for keyword in top_keywords:
    places = data.loc[data['키워드'].apply(lambda x: keyword in x), '관광지'].unique().tolist()
    keyword_to_places[keyword] = places

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame([(keyword, ', '.join(places)) for keyword, places in keyword_to_places.items()], 
                         columns=['키워드', '관광지 리스트'])

# 결과 저장
result_file_path = './data/서브워드토큰화_관광지리스트.csv'
result_df.to_csv(result_file_path, index=False)

# 결과 파일 경로 출력
result_file_path

## 2. 감정분석

### 리뷰_표제어추출

1: 긍정적인 감정
0: 부정적인 감정

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

# 데이터 파일 로드 (파일 경로를 정확히 지정)
file_path = '/content/형태소_sampled.csv'
data = pd.read_csv(file_path)

# 사전 학습된 모델과 토크나이저 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        inputs = self.tokenizer(
            review,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
        }

# 감정 예측 함수 정의
def predict_sentiment(model, dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # 배치 처리 중에는 필요하지 않은 계산 생략
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            predictions.extend(preds)

    return predictions

# 모델을 CPU 또는 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 표제어추출 데이터셋 준비 및 감정 분석 (최적화 적용)
lemma_reviews = data['리뷰_표제어추출'].dropna().tolist()
lemma_dataset = ReviewDataset(lemma_reviews, tokenizer, max_length=128)
lemma_dataloader = DataLoader(lemma_dataset, batch_size=32, shuffle=False, num_workers=4)  # 배치 크기 증가, 멀티프로세싱 사용

lemma_predictions = predict_sentiment(model, lemma_dataloader)

# 예측 결과를 원래 데이터프레임에 추가
data.loc[data['리뷰_표제어추출'].notna(), 'Sentiment_Lemma'] = lemma_predictions

# 결과 저장
data.to_csv('sentiment_analysis_lemma.csv', index=False)

print("Sentiment analysis completed and results saved to 'sentiment_analysis_lemma.csv'")

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 데이터프레임의 첫 5행 확인
data.head()

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 관광지 이름 별로 Sentiment 0과 1의 개수를 세는 코드
sentiment_counts = data.groupby('관광지')['Sentiment_Lemma'].value_counts().unstack().fillna(0)


# 감정 분포 확인
sentiment_distribution = data['Sentiment_Lemma'].value_counts()
print(sentiment_distribution)

# 결과 확인
sentiment_counts

### 리뷰_불용어제거
1: 긍정적인 감정 0: 부정적인 감정

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

# 데이터 파일 로드 (파일 경로를 정확히 지정)
file_path = '/content/형태소_sampled.csv'
data = pd.read_csv(file_path)

# 사전 학습된 모델과 토크나이저 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        inputs = self.tokenizer(
            review,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
        }

# 감정 예측 함수 정의
def predict_sentiment(model, dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # 배치 처리 중에는 필요하지 않은 계산 생략
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            predictions.extend(preds)

    return predictions

# 모델을 CPU 또는 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 표제어추출 데이터셋 준비 및 감정 분석 (최적화 적용)
lemma_reviews = data['리뷰_불용어제거'].dropna().tolist()
lemma_dataset = ReviewDataset(lemma_reviews, tokenizer, max_length=128)
lemma_dataloader = DataLoader(lemma_dataset, batch_size=32, shuffle=False, num_workers=4)  # 배치 크기 증가, 멀티프로세싱 사용

lemma_predictions = predict_sentiment(model, lemma_dataloader)

# 예측 결과를 원래 데이터프레임에 추가
data.loc[data['리뷰_불용어제거'].notna(), 'Sentiment_Lemma'] = lemma_predictions

# 결과 저장
data.to_csv('sentiment_analysis_lemma_2.csv', index=False)

print("Sentiment analysis completed and results saved to 'sentiment_analysis_lemma.csv'")

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma_2.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 데이터프레임의 첫 5행 확인
data.head()

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma_2.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 관광지 이름 별로 Sentiment 0과 1의 개수를 세는 코드
sentiment_counts = data.groupby('관광지')['Sentiment_Lemma'].value_counts().unstack().fillna(0)


# 감정 분포 확인
sentiment_distribution = data['Sentiment_Lemma'].value_counts()
print(sentiment_distribution)

# 결과 확인
sentiment_counts

### 리뷰 표제어 추출값으로 확률 값 확인

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn.functional as F  # Softmax를 위해 필요

# 데이터 파일 로드 (파일 경로를 정확히 지정)
file_path = '/content/형태소_sampled.csv'
data = pd.read_csv(file_path)

# 사전 학습된 모델과 토크나이저 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        inputs = self.tokenizer(
            review,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
        }

# 감정 예측 함수 정의 (확률로 반환)
def predict_sentiment(model, dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # 소프트맥스를 사용해 확률로 변환
            probs = F.softmax(logits, dim=-1)
            positive_probs = probs[:, 1].cpu().numpy()  # 긍정 클래스(1)의 확률을 가져옴
            predictions.extend(positive_probs)

    return predictions

# 모델을 CPU 또는 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 표제어추출 데이터셋 준비 및 감정 분석 (최적화 적용)
lemma_reviews = data['리뷰_표제어추출'].dropna().tolist()
lemma_dataset = ReviewDataset(lemma_reviews, tokenizer, max_length=128)
lemma_dataloader = DataLoader(lemma_dataset, batch_size=32, shuffle=False, num_workers=4)  # 배치 크기 증가, 멀티프로세싱 사용

lemma_predictions = predict_sentiment(model, lemma_dataloader)

# 예측 결과를 원래 데이터프레임에 추가
data.loc[data['리뷰_표제어추출'].notna(), 'Sentiment_Lemma'] = lemma_predictions

# 결과 저장
data.to_csv('sentiment_analysis_lemma_percent.csv', index=False)

print("Sentiment analysis completed and results saved to 'sentiment_analysis_lemma.csv'")

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma_percent.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 데이터프레임의 첫 5행 확인
data.head()

In [ ]:
import matplotlib.pyplot as plt

# 데이터 파일 로드
file_path = '/content/sentiment_analysis_lemma_percent.csv'
data = pd.read_csv(file_path)

# Sentiment_Lemma 확률 값 추출
sentiment_probs = data['Sentiment_Lemma'].dropna()  # NaN 값 제거

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(sentiment_probs, bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Sentiment_Lemma Probabilities')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.grid(True)

# 그래프 표시
plt.show()

### 모델 고도화

BERT 모델 바꿔보기

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn.functional as F

# 데이터 파일 로드
file_path = '/content/형태소_sampled.csv'
data = pd.read_csv(file_path)

# 모델과 토크나이저 로드
model_name = "klue/roberta-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        inputs = self.tokenizer(
            review,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
        }

def predict_sentiment(model, dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)
            positive_probs = probs[:, 1].cpu().numpy()
            predictions.extend(positive_probs)

    return predictions

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

lemma_reviews = data['리뷰_표제어추출'].dropna().tolist()
lemma_dataset = ReviewDataset(lemma_reviews, tokenizer, max_length=128)
lemma_dataloader = DataLoader(lemma_dataset, batch_size=32, shuffle=False, num_workers=4)

lemma_predictions = predict_sentiment(model, lemma_dataloader)

data.loc[data['리뷰_표제어추출'].notna(), 'Sentiment_Lemma'] = lemma_predictions
data.to_csv('sentiment_analysis_lemma_percent.csv', index=False)

print("Sentiment analysis completed and results saved to 'sentiment_analysis_lemma_percent.csv'")

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma_percent.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 데이터프레임의 첫 5행 확인
data.head()

In [ ]:
# 데이터 파일 로드
file_path = '/content/sentiment_analysis_lemma_percent.csv'
data = pd.read_csv(file_path)

# Sentiment_Lemma 확률 값 추출
sentiment_probs = data['Sentiment_Lemma'].dropna()  # NaN 값 제거

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(sentiment_probs, bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Sentiment_Lemma Probabilities')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.grid(True)

# 그래프 표시
plt.show()

### 모델 앙상블 (최종 선택 방법)

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn.functional as F

# 데이터 파일 로드
file_path = '/content/형태소.csv'
data = pd.read_csv(file_path)

# 모델과 토크나이저 로드
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

# 두 개의 BERT 모델 로드
model1 = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model2 = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 모델을 CPU 또는 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)
model2.to(device)

class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        inputs = self.tokenizer(
            review,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
        }

def predict_sentiment(model, dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)
            positive_probs = probs[:, 1].cpu().numpy()
            predictions.extend(positive_probs)

    return predictions

# 데이터셋 준비
lemma_reviews = data['리뷰_불용어제거'].dropna().tolist()
lemma_dataset = ReviewDataset(lemma_reviews, tokenizer, max_length=128)
lemma_dataloader = DataLoader(lemma_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델 1 예측
print("Predicting with Model 1...")
lemma_predictions_model1 = predict_sentiment(model1, lemma_dataloader)

# 모델 2 예측
print("Predicting with Model 2...")
lemma_predictions_model2 = predict_sentiment(model2, lemma_dataloader)

# 두 모델의 예측을 평균화
ensemble_predictions = [(p1 + p2) / 2 for p1, p2 in zip(lemma_predictions_model1, lemma_predictions_model2)]

# 예측 결과를 원래 데이터프레임에 추가
data.loc[data['리뷰_불용어제거'].notna(), 'Sentiment_Lemma'] = ensemble_predictions

# 결과 저장
data.to_csv('sentiment_analysis_lemma_ensemble_bert.csv', index=False)

print("Sentiment analysis completed with ensemble model and results saved to 'sentiment_analysis_lemma_ensemble.csv'")

In [ ]:
# 긴 텍스트가 잘리지 않도록 설정
pd.set_option('display.max_colwidth', None)

# 파일 경로
file_path = '/content/sentiment_analysis_lemma_ensemble_bert.csv'

# CSV 파일을 데이터프레임으로 로드
data = pd.read_csv(file_path)

# 데이터프레임의 첫 5행 확인
data.head(20)

In [ ]:
import matplotlib.pyplot as plt

# 데이터 파일 로드
file_path = '/content/sentiment_analysis_lemma_ensemble_bert.csv'
data = pd.read_csv(file_path)

# Sentiment_Lemma 확률 값 추출
sentiment_probs = data['Sentiment_Lemma'].dropna()  # NaN 값 제거

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(sentiment_probs, bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Sentiment_Lemma Probabilities')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.grid(True)

# 그래프 표시
plt.show()